<a href="https://colab.research.google.com/github/snvssk/data298A/blob/saidev/Fetch_Satelliteimage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import multiprocessing

import os
import queue
import shutil
import threading
from PIL import Image
from datetime import datetime
from urllib import request

In [ ]:
class MapDownloader(object):
    def __init__(self, lat_start, lng_start, lat_end, lng_end, zoom=12, tile_size=256):
        #self.tile_server = 'https://mts1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
        #self.tile_server = 'http://tile.openstreetmap.org/{z}/{x}/{y}.png'
        self.tile_server = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

        self.lat_start = lat_start
        self.lng_start = lng_start
        self.lat_end = lat_end
        self.lng_end = lng_end
        self.zoom = zoom
        self.tile_size = tile_size

        self.q = queue.Queue()
        self.num_worker = multiprocessing.cpu_count() - 1

        self._generate_xy_point()

    def _generate_xy_point(self):
        self._x_start, self._y_start = self._convert_latlon_to_xy(self.lat_start, self.lng_start)
        self._x_end, self._y_end = self._convert_latlon_to_xy(self.lat_end, self.lng_end)

    def _convert_latlon_to_xy(self, lat, lng):
        tiles_count = 1 << self.zoom

        point_x = (self.tile_size / 2 + lng * self.tile_size / 360.0) * tiles_count // self.tile_size
        sin_y = math.sin(lat * (math.pi / 180.0))
        point_y = ((self.tile_size / 2) + 0.5 * math.log((1 + sin_y) / (1 - sin_y)) *
                   -(self.tile_size / (2 * math.pi))) * tiles_count // self.tile_size

        return int(point_x), int(point_y)

    def _fetch_worker(self):
        while True:
            item = self.q.get()
            if item is None:
                break

            idx, url, current_tile = item
            print('Fetching #{} of {}: {}'.format(idx, self.q_size, url))
            request.urlretrieve(url, current_tile)

            self.q.task_done()

    def write_into(self, filename):
        # create temp dir
        directory = os.path.abspath('./{}'.format(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")))
        if not os.path.exists(directory):
            os.makedirs(directory)

        # generate source list
        idx = 1
        for x in range(0, self._x_end + 1 - self._x_start):
            for y in range(0, self._y_end + 1 - self._y_start):
                url = self.tile_server.format(
                    x=str(self._x_start + x), y=str(self._y_start + y), z=str(self.zoom))
                current_tile = os.path.join(directory, 'tile-{}_{}_{}.png'.format(
                    str(self._x_start + x), str(self._y_start + y), str(self.zoom)))
                self.q.put((idx, url, current_tile))
                idx += 1

        # stop workers
        for i in range(self.num_worker):
            self.q.put(None)

        # start fetching tile using multithread to speed up process
        self.q_size = self.q.qsize()

        threads = []
        for i in range(self.num_worker):
            t = threading.Thread(target=self._fetch_worker)
            t.start()
            threads.append(t)

        for t in threads:
            t.join()

        # combine image into single
        width, height = 256 * (self._x_end + 1 - self._x_start), 256 * (self._y_end + 1 - self._y_start)
        map_img = Image.new('RGB', (width, height))

        for x in range(0, self._x_end + 1 - self._x_start):
            for y in range(0, self._y_end + 1 - self._y_start):
                current_tile = os.path.join(directory, 'tile-{}_{}_{}.png'.format(
                    str(self._x_start + x), str(self._y_start + y), str(self.zoom)))
                im = Image.open(current_tile)
                map_img.paste(im, (x * 256, y * 256))

        map_img.save(filename)

        # remove temp dir
        shutil.rmtree(directory)

In [ ]:
def main():
    try:
        md = MapDownloader(-6.256524, 107.170208, -6.292112, 107.242934, zoom=18)
        #md = MapDownloader(37.408990,-121.897890, 37.408990,-121.897890, zoom=19)
        
        md.write_into('myhome.png')

        print("The map has successfully been created")
    except Exception as e:
        print(
            "Could not generate the image - try adjusting the zoom level and checking your coordinates. Cause: {}".format(
                e))


if __name__ == '__main__':
    main()

Fetching #1 of 1513: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/18/135636/209110
Fetching #2 of 1513: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/18/135637/209110
Fetching #3 of 1513: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/18/135638/209110
Fetching #4 of 1513: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/18/135639/209110
Fetching #5 of 1513: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/18/135640/209110
Fetching #6 of 1513: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/18/135641/209110
Fetching #7 of 1513: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/18/135642/209110
Fetching #8 of 1513: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/18/135643/209110
Fetching #9 of 1513: https://server.arcg

In [14]:

!apt-get update
!pip install selenium
!apt install firefox-geckodriver
!cp -r /usr/lib/geckodriver /usr/bin
!cp -r /usr/lib/firefox /usr/bin

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Looking in indexes: https://pypi.org/simple, https:

In [ ]:
#Watipaso Mulwafu
#https://github.com/thewati
#https://medium.com/@watipasomulwafu

#import all librabries
import csv
import folium
import io
from PIL import Image
import IPython

#open csv file containing latitudes and longitudes to read through
#reader = csv.reader(open('coordinates.csv', 'r'), delimiter=',')
lat = '37.33670991477899'
lon = '-121.87847503182728'
reader = [[lat,lon,'sjsu']]
#set your access token obtained from your mapbox account. See instructions on GitHub for more
mapboxAccessToken = 'pk.eyJ1Ijoic252c3NhaWt1bWFyIiwiYSI6ImNsNnBpMzVhdjA0NXUzY21uMGVzcjY2OXoifQ.vrpbgJTSoha1uPI1RlN_KA'
#set tile set
mapboxTilesetId = 'mapbox.satellite'
#print(reader)
#loop through all coordinates in csv file
for row in reader:
    #extracting each cell in row individually
    key = (row[0], row[1], row[2])
    #print(key)

    m = folium.Map(
        location=[row[0], row[1]],
        zoom_start=20,
        tiles='https://api.tiles.mapbox.com/v4/' + mapboxTilesetId + '/{z}/{x}/{y}.png?access_token=' + mapboxAccessToken,attr='mapbox.com')
    m.save(row[2] + '.html') #save as html file
    # take screenshot of html page and save as .png file
    #print(m)
    #img_data = m._to_png(4)
    #img = Image.open(io.BytesIO(img_data))
    #img.save(row[2] + '.png')
    #from IPython.display import IFrame
    #IFrame(src=row[2] + '.html', width=300, height=500)
    
IPython.display.HTML(row[2] + '.html')

In [8]:
!curl -g "https://api.mapbox.com/styles/v1/mapbox/streets-v11/static/-122.4241,37.78,15.25,0,60/400x400?access_token=pk.eyJ1Ijoic252c3NhaWt1bWFyIiwiYSI6ImNsNnBpMzVhdjA0NXUzY21uMGVzcjY2OXoifQ.vrpbgJTSoha1uPI1RlN_KA" --output example-mapbox-static-1.png


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55820  100 55820    0     0   119k      0 --:--:-- --:--:-- --:--:--  119k


In [10]:
!curl "https://api.mapbox.com/v4/mapbox.mapbox-streets-v8/1/0/0.mvt?access_token=pk.eyJ1Ijoic252c3NhaWt1bWFyIiwiYSI6ImNsNnBpMzVhdjA0NXUzY21uMGVzcjY2OXoifQ.vrpbgJTSoha1uPI1RlN_KA" --output example-vector.png



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39618  100 39618    0     0   132k      0 --:--:-- --:--:-- --:--:--  132k


In [11]:
! curl "https://api.mapbox.com/v4/mapbox.satellite/1/0/0@2x.jpg90?access_token=pk.eyJ1Ijoic252c3NhaWt1bWFyIiwiYSI6ImNsNnBpMzVhdjA0NXUzY21uMGVzcjY2OXoifQ.vrpbgJTSoha1uPI1RlN_KA" --output example-rastor.tiff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51212  100 51212    0     0   163k      0 --:--:-- --:--:-- --:--:--  163k


In [23]:
!curl https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/-121.8975,37.4091,19,0/600x500?access_token=pk.eyJ1Ijoic252c3NhaWt1bWFyIiwiYSI6ImNsNnBpMzVhdjA0NXUzY21uMGVzcjY2OXoifQ.vrpbgJTSoha1uPI1RlN_KA --output example-satellite.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 83201  100 83201    0     0   130k      0 --:--:-- --:--:-- --:--:--  130k


In [5]:
!curl https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/-122.0252,37.4179,17,0/256x256@2x?access_token=pk.eyJ1Ijoic252c3NhaWt1bWFyIiwiYSI6ImNsNnBpMzVhdjA0NXUzY21uMGVzcjY2OXoifQ.vrpbgJTSoha1uPI1RlN_KA  --output example-satellite-image.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 72101  100 72101    0     0   181k      0 --:--:-- --:--:-- --:--:--  181k
